# Part 3.5 - Improvements

> See `part3_5_pipeline.py` for code to run the training.

We will try different transformers model architecture and training mode to find the best method for sentiment analysis:


| **Model**   | **Architecture**  | **Training Mode**                | **Description**                                                     |
|-------------|-------------------|----------------------------------|---------------------------------------------------------------------|
| **RoBERTa** | Encoder Only      | Pretrained                       | The original pretrained RoBERTa model, trained on a general corpus. |
|             |                   | Fine-Tuned on Sentiment Analysis | RoBERTa fine-tuned on sentiment analysis.                           |
| **GPT-2**   | Decoder Only      | Pretrained                       | The original pretrained GPT-2 model, trained on a general corpus.   |
|             |                   | Fine-Tuned on Sentiment Analysis | GPT-2 fine-tuned on sentiment analysis.                             |
| **T5**      | Encoder + Decoder | Pretrained                       | The original T5 model, trained on a general corpus.                 |



In [2]:
from utils.analytics import load_tensorboard_logs_from_huggingface_trainer

results_df = load_tensorboard_logs_from_huggingface_trainer("part3_5_logs/")


# Test accuracy that was printed to the terminal:
test_accuracies = {
    "roberta": 0.8583489656448364,
    "roberta-finetuned": 0.8584173729742681,
    "gpt2-finetuned": 0.8536585569381714,
    "gpt2": 0.8508442640304565,
    "t5": 0.8424015045166016,
}

results_df["test_acc"] = results_df["model"].map(test_accuracies)
results_df = results_df[["model", "val_acc", "val_loss", "test_acc", "filename"]]
results_df = results_df.sort_values("val_acc", ascending=False).reset_index(drop=True)

results_df

,model,val_acc,val_loss,test_acc,filename
0,roberta-finetuned,0.888368,0.356488,0.858417,events.out.tfevents.1730547046.yuriarch.51191.0
1,gpt2-finetuned,0.881801,0.332477,0.853659,events.out.tfevents.1730720697.yuriarch.3740.0
2,roberta,0.873358,0.341587,0.858349,events.out.tfevents.1730785167.yuriarch.1754.0
3,gpt2,0.866792,0.333615,0.850844,events.out.tfevents.1730824378.1e14d39e96a8.15...
4,t5,0.854597,0.361825,0.842402,events.out.tfevents.1730885598.776fcca9ca6f.65...
